This dataset classifies people described by a set of attributes as good or bad credit risks.
There is no data cleaning. 

You should try to predict whether a person's credit is good or bad.


In [111]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from graphviz import Source
from sklearn.tree import export_graphviz

!pip install graphviz

In [112]:
from sklearn.datasets import fetch_openml
creditG = fetch_openml('credit-g', version=1,as_frame=True)

In [113]:
df = creditG['frame']

df is the dataframe you need to make classification.'class' column is the target variable. You can find more info about this dataset at https://www.openml.org/d/31


In [114]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,...,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad


Build two regression tree models to predict whether a person has a bad or good credit....

The first model should have  max_depth=2. The second model should have  max_depth=4.

Build confusion matrix for both models. Which one seems to be better?

In [115]:
df.isnull().sum()

checking_status           0
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          0
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
class                     0
dtype: int64

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
checking_status           1000 non-null category
duration                  1000 non-null float64
credit_history            1000 non-null category
purpose                   1000 non-null category
credit_amount             1000 non-null float64
savings_status            1000 non-null category
employment                1000 non-null category
installment_commitment    1000 non-null float64
personal_status           1000 non-null category
other_parties             1000 non-null category
residence_since           1000 non-null float64
property_magnitude        1000 non-null category
age                       1000 non-null float64
other_payment_plans       1000 non-null category
housing                   1000 non-null category
existing_credits          1000 non-null float64
job                       1000 non-null category
num_dependents            1000 non-null float64
own_telephone      

In [117]:
df['own_telephone'].value_counts()

none    596
yes     404
Name: own_telephone, dtype: int64

In [118]:
df['class'].value_counts()

good    700
bad     300
Name: class, dtype: int64

In [119]:
telephone_mapping = {"none": 0, "yes": 1}

In [120]:
df['own_telephone'] = df['own_telephone'].map(telephone_mapping)

In [121]:
df = df.drop(['checking_status','credit_history','purpose','savings_status','employment','personal_status','other_parties','property_magnitude','other_payment_plans','housing','job','foreign_worker'], axis = 1)

In [122]:
df.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,own_telephone,class
0,6.0,1169.0,4.0,4.0,67.0,2.0,1.0,1,good
1,48.0,5951.0,2.0,2.0,22.0,1.0,1.0,0,bad
2,12.0,2096.0,2.0,3.0,49.0,1.0,2.0,0,good
3,42.0,7882.0,2.0,4.0,45.0,1.0,2.0,0,good
4,24.0,4870.0,3.0,4.0,53.0,2.0,2.0,0,bad


In [123]:
X = df.drop('class', axis = 1)
y = df['class']

In [124]:
X_train2,X_test2,y_train2,y_test2 = train_test_split(X,y,test_size=0.2,random_state=42)

In [125]:
X_train4,X_test4,y_train4,y_test4 = train_test_split(X,y,test_size=0.2,random_state=42)

In [126]:
from sklearn.tree import DecisionTreeClassifier

In [127]:
dtree2 = DecisionTreeClassifier(max_depth=2)

In [128]:
dtree4 = DecisionTreeClassifier(max_depth=4)

In [129]:
dtree2.fit(X_train2,y_train2)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [130]:
dtree4.fit(X_train4,y_train4)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [131]:
predictions2 = dtree2.predict(X_test2)
predictions2

array(['good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'bad', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'bad', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'bad',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'bad', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'bad', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'bad', 'good', 'good',
       'good

In [132]:
predictions4 = dtree4.predict(X_test4)
predictions4

array(['good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'bad', 'good', 'good', 'good', 'bad', 'bad', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'bad',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'bad',
       'good', 'good', 'good', 'good', 'bad', 'good', 'bad', 'good',
       'good', 'good', 'bad', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'bad', 'good', 'good',
       'good', 

In [133]:
results2 = pd.DataFrame({
    'actual': y_test2,
    'prediction': predictions2
})
results2

,actual,prediction
521,bad,good
737,good,good
740,good,good
660,good,good
411,good,good
...,...,...
408,good,good
332,bad,bad
208,good,good
613,good,good


In [134]:
results4 = pd.DataFrame({
    'actual': y_test4,
    'prediction': predictions4
})
results4

,actual,prediction
521,bad,good
737,good,good
740,good,good
660,good,good
411,good,good
...,...,...
408,good,good
332,bad,good
208,good,good
613,good,good


In [135]:
from sklearn.metrics import confusion_matrix

In [136]:
confusion_matrix(y_test2,predictions2)

array([[  6,  53],
       [  2, 139]])

In [137]:
confusion_matrix(y_test4,predictions4)

array([[  8,  51],
       [  6, 135]])

In [138]:
results2[(results2['actual']=='good')&(results2['prediction']=='good')]

,actual,prediction
737,good,good
740,good,good
660,good,good
411,good,good
678,good,good
...,...,...
829,good,good
408,good,good
208,good,good
613,good,good


In [139]:
results4[(results4['actual']=='good')&(results4['prediction']=='good')]

,actual,prediction
737,good,good
740,good,good
660,good,good
411,good,good
678,good,good
...,...,...
942,good,good
749,good,good
408,good,good
208,good,good


In [140]:
from sklearn.metrics import classification_report 

In [141]:
print(classification_report(y_test2,predictions2))

              precision    recall  f1-score   support

         bad       0.75      0.10      0.18        59
        good       0.72      0.99      0.83       141

    accuracy                           0.73       200
   macro avg       0.74      0.54      0.51       200
weighted avg       0.73      0.72      0.64       200



In [142]:
print(classification_report(y_test4,predictions4))

              precision    recall  f1-score   support

         bad       0.57      0.14      0.22        59
        good       0.73      0.96      0.83       141

    accuracy                           0.71       200
   macro avg       0.65      0.55      0.52       200
weighted avg       0.68      0.71      0.65       200



The model with max_depth=2 was better as seen with higher accuracy.